WARNING: TESTINGS ARE MISSING FOR THE DATASET CLASS

# WNDataset test

Now that we have one segmentation item (Image + Mask) working, we need something to group N segmentation items and supply them to the learner. Some frameworks separate dataset and dataloader. We will do it in one step.

In [206]:
%load_ext autoreload
%autoreload 2

from time import sleep

import rasterio as rio
from tqdm.notebook import tqdm

from waternet.wndataset import WNDataSet
from waternet.wnsegmantationitem import WNSegmentationItem

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [207]:
dset = WNDataSet.from_masks(
    masks_path='d:/temp/22KEV/',
    out_shape=(5490, 5490),
    patch_size=(512, 512),
    step=262,
    pattern='*20181*.tif',
)

print(dset)

Matching imgs/masks:   0%|          | 0/11 [00:00<?, ?it/s]

WNDataset instance with 11 images
Loaded: 0 items
Empty: 11 items
Loading: 0 items



If we try to access one item, without proper initialization...

In [208]:
try:
    dset[0]
except Exception as e:
    print(e)

Bands not set. Set desired bands first.


We need to specify which bands from the StacImage will be used in the DataSet

In [209]:
dset.bands = ['B11', 'B12']

Now, if we try to access one "item"

In [210]:
dset[0]

Loading image 1 in background


(array([[[0.375 , 0.3749, 0.3972, ..., 0.3526, 0.3675, 0.3712],
         [0.3804, 0.3745, 0.3759, ..., 0.3573, 0.3765, 0.3803],
         [0.3803, 0.3797, 0.3672, ..., 0.2881, 0.3092, 0.3029],
         ...,
         [0.3806, 0.3799, 0.3817, ..., 0.3647, 0.3622, 0.356 ],
         [0.3816, 0.3777, 0.3817, ..., 0.3665, 0.3665, 0.3627],
         [0.3782, 0.3733, 0.3766, ..., 0.3631, 0.3673, 0.3692]],
 
        [[0.2914, 0.2926, 0.3144, ..., 0.2405, 0.257 , 0.2683],
         [0.2979, 0.2869, 0.2848, ..., 0.2606, 0.274 , 0.2761],
         [0.2938, 0.2858, 0.2714, ..., 0.2277, 0.2452, 0.2478],
         ...,
         [0.3258, 0.3149, 0.3132, ..., 0.3016, 0.3004, 0.295 ],
         [0.3242, 0.3132, 0.3136, ..., 0.3038, 0.3025, 0.2977],
         [0.3235, 0.3117, 0.3132, ..., 0.3007, 0.3053, 0.3053]]],
       dtype=float32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]

In [211]:
print(dset)

WNDataset instance with 11 images
Loaded: 1 items
Empty: 9 items
Loading: 1 items



In [212]:
dset.loaded_status

{0: 'Loaded',
 1: 'Loading',
 2: 'Empty',
 3: 'Empty',
 4: 'Empty',
 5: 'Empty',
 6: 'Empty',
 7: 'Empty',
 8: 'Empty',
 9: 'Empty',
 10: 'Empty'}

One should note that the length of a DataSet is given by the number of <b>Patches</b> and not the number of "images". Each item that we iterate through is a small patch.

In [213]:
print(len(dset))
print(dset[0][0].shape, dset[0][1].shape)

4400
(2, 512, 512) (512, 512)


Testing a workload...

In [214]:
for patch, mask in tqdm(dset):
    patch.mean()
    sleep(0.1)

  0%|          | 0/4400 [00:00<?, ?it/s]

Loaded sucessfully
Cleared image 0
Loading image 2 in background
Loaded sucessfully
Cleared image 1
Loading image 3 in background
Loaded sucessfully
Cleared image 2
Loading image 4 in background
Loaded sucessfully
Cleared image 3
Loading image 5 in background
Loaded sucessfully
Cleared image 4
Loading image 6 in background
Loaded sucessfully
Cleared image 5
Loading image 7 in background
Loaded sucessfully
Cleared image 6
Loading image 8 in background
Loaded sucessfully
Cleared image 7
Loading image 9 in background
Loaded sucessfully
Cleared image 8
Loading image 10 in background
Loaded sucessfully
Cleared image 9
Loading image 0 in background
Loaded sucessfully


## Putting it into a DataLoader

In [215]:
from fastai.data.load import DataLoader

In [216]:
dl = DataLoader(dataset=dset)
b = dl.one_batch()
print(b[0].shape, b[1].shape)

Cleared image 10
Loading image 1 in background
torch.Size([2, 512, 512]) torch.Size([512, 512])


In [217]:
len(dl)

4400

Note that we don't have here the dimension regarding the batch. That's because BatchSize was not specified.

In [218]:
dl = DataLoader(dataset=dset, bs=16)
b = dl.one_batch()
print(b[0].shape, b[1].shape)

torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])


In [219]:
len(dl)

275

In [220]:
for b in tqdm(dl):
    print(b[0].shape, b[1].shape)
    sleep(0.5)

  0%|          | 0/275 [00:00<?, ?it/s]

torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) torch.Size([16, 512, 512])
torch.Size([16, 2, 512, 512]) t

After some time I finally understand the do_batch. Looking below, we can see that it creates a batch from a list of items. In our case, it will "collate" the items without any further processing. 

In [221]:
dl.do_batch([dset[0], dset[1]])

Cleared image 10
Loading image 1 in background
Loaded sucessfully


(tensor([[[[0.3750, 0.3749, 0.3972,  ..., 0.3526, 0.3675, 0.3712],
           [0.3804, 0.3745, 0.3759,  ..., 0.3573, 0.3765, 0.3803],
           [0.3803, 0.3797, 0.3672,  ..., 0.2881, 0.3092, 0.3029],
           ...,
           [0.3806, 0.3799, 0.3817,  ..., 0.3647, 0.3622, 0.3560],
           [0.3816, 0.3777, 0.3817,  ..., 0.3665, 0.3665, 0.3627],
           [0.3782, 0.3733, 0.3766,  ..., 0.3631, 0.3673, 0.3692]],
 
          [[0.2914, 0.2926, 0.3144,  ..., 0.2405, 0.2570, 0.2683],
           [0.2979, 0.2869, 0.2848,  ..., 0.2606, 0.2740, 0.2761],
           [0.2938, 0.2858, 0.2714,  ..., 0.2277, 0.2452, 0.2478],
           ...,
           [0.3258, 0.3149, 0.3132,  ..., 0.3016, 0.3004, 0.2950],
           [0.3242, 0.3132, 0.3136,  ..., 0.3038, 0.3025, 0.2977],
           [0.3235, 0.3117, 0.3132,  ..., 0.3007, 0.3053, 0.3053]]],
 
 
         [[[0.3411, 0.3001, 0.2992,  ..., 0.2209, 0.2147, 0.2037],
           [0.3117, 0.2855, 0.3014,  ..., 0.2377, 0.2461, 0.2085],
           [0.2700, 0

In [222]:
dset.clear()

## Creating a validation dataset

To create a learner from Fastai we need to have a Dataloader for the training set and another one for the validation set. To solve that, we will create a new DataSet from another scene, to serve as validation.

In [223]:
segitem = WNSegmentationItem.from_mask(
    mask_path='d:/temp/22KEV/S2B_MSIL2A_20210221T133219_R081_T22KEV_watermask.tif',
    shape=(5490, 5490),
    step=262,
)

valid_dset = WNDataSet([segitem]) # , segitem])
print(valid_dset)
len(valid_dset)

WNDataset instance with 1 images
Loaded: 0 items
Empty: 1 items
Loading: 0 items



400

In [224]:
valid_dset.bands = ['B11', 'B12']

In [225]:
# test the iteration on the valid_dset

for patch, mask in tqdm(valid_dset):
    patch.mean()
    sleep(0.1)

  0%|          | 0/400 [00:00<?, ?it/s]

Loaded sucessfully


In [226]:
valid_dset.loaded_status

{0: 'Loaded'}

## Creating a Learner from these datasets

In [227]:
from fastai.vision.all import *

In [228]:
# create the dataloader for the validation dataset
valid_dl = DataLoader(valid_dset, bs=16)

dls = DataLoaders(dl, valid_dl)

In [229]:
# Dataloaders has 2 dataloaders, and it assigns automatically the training and the validation sets
print(len(dls))

2


In [230]:
print(dls.valid_ds, dls.train_ds)

WNDataset instance with 1 images
Loaded: 1 items
Empty: 0 items
Loading: 0 items
 WNDataset instance with 11 images
Loaded: 1 items
Empty: 10 items
Loading: 0 items



In [231]:
def loss_fn(pred, targ):
    return torch.nn.functional.cross_entropy(pred, targ.squeeze(1).type(torch.long))

In [232]:
learner = unet_learner(dls, arch=resnet18, normalize=False, n_in=2, n_out=3, loss_func=loss_fn)

d:\Programs\miniconda3\envs\fastai\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Programs\miniconda3\envs\fastai\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [233]:
b, targ = dl.one_batch()

In [234]:
b.dtype

torch.float32

In [235]:
learner.model(b)

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1644167168 bytes.

In [ ]:
learner.cuda()

DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05

In [ ]:
learner.lr_find()

RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 6.00 GiB total capacity; 4.50 GiB already allocated; 0 bytes free; 5.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
class DummyIterator:
    def __iter__(self):
        yield from range(11)

In [ ]:
di = DummyIterator()

In [ ]:
next(iter(di))

0

In [ ]:
next(iter(dset))

(array([[[0.375 , 0.3749, 0.3972, ..., 0.3526, 0.3675, 0.3712],
         [0.3804, 0.3745, 0.3759, ..., 0.3573, 0.3765, 0.3803],
         [0.3803, 0.3797, 0.3672, ..., 0.2881, 0.3092, 0.3029],
         ...,
         [0.3806, 0.3799, 0.3817, ..., 0.3647, 0.3622, 0.356 ],
         [0.3816, 0.3777, 0.3817, ..., 0.3665, 0.3665, 0.3627],
         [0.3782, 0.3733, 0.3766, ..., 0.3631, 0.3673, 0.3692]],
 
        [[0.2914, 0.2926, 0.3144, ..., 0.2405, 0.257 , 0.2683],
         [0.2979, 0.2869, 0.2848, ..., 0.2606, 0.274 , 0.2761],
         [0.2938, 0.2858, 0.2714, ..., 0.2277, 0.2452, 0.2478],
         ...,
         [0.3258, 0.3149, 0.3132, ..., 0.3016, 0.3004, 0.295 ],
         [0.3242, 0.3132, 0.3136, ..., 0.3038, 0.3025, 0.2977],
         [0.3235, 0.3117, 0.3132, ..., 0.3007, 0.3053, 0.3053]]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..

In [ ]:
import rasterio as rio

In [ ]:
from waternet.wnstacimage import WNStacImage
import planetary_computer as pc

In [ ]:
img = WNStacImage.from_tile(
    tile='22KEV',
    str_date="2022-08-25",
    shape=(5490, 5490)
)

In [ ]:
img['B01']

[<Asset href=https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/K/EV/2022/08/25/S2B_MSIL2A_20220825T133149_N0400_R081_T22KEV_20220826T055538.SAFE/GRANULE/L2A_T22KEV_A028564_20220825T133432/IMG_DATA/R60m/T22KEV_20220825T133149_B01_60m.tif>]

In [ ]:
ds = rio.open(pc.sign(img['B01'][0].href), dtype='uint8')

In [ ]:
rio.open?

Signature:
rio.open(
    fp,
    mode='r',
    driver=None,
    width=None,
    height=None,
    count=None,
    crs=None,
    transform=None,
    dtype=None,
    nodata=None,
    sharing=False,
    **kwargs,
)
Docstring:
Open a dataset for reading or writing.

The dataset may be located in a local file, in a resource located by
a URL, or contained within a stream of bytes.

In read ('r') or read/write ('r+') mode, no keyword arguments are
required: these attributes are supplied by the opened dataset.

In write ('w' or 'w+') mode, the driver, width, height, count, and dtype
keywords are strictly required.

Parameters
----------
fp : str, file object or pathlib.Path object
    A filename or URL, a file object opened in binary ('rb') mode,
    or a Path object.
mode : str, optional
    'r' (read, the default), 'r+' (read/write), 'w' (write), or
    'w+' (write/read).
driver : str, optional
    A short format driver name (e.g. "GTiff" or "JPEG") or a list of
    such names (see GDAL docs 

In [ ]:
(ds.read().astype('float32')/20000).dtype

dtype('float32')

In [ ]:
        "mask_path": "d:/temp/22KEV/S2A_MSIL2A_20180123T133221_R081_T22KEV_watermask.tif",
        "stac_tile": "22KEV",
        "stac_date": "2022-08-25",
        "stac_name": "S2B_MSIL2A_20220825T133149_R081_T22KEV_20220826T055538",
        "stac_shape": (5490, 5490),
        "stac_bands": ["B01", "B09"],
    }
    return fixtures


@pytest.fixture(scope="session")
def stac_img(test_data):
    """Load a stac_img that will be used accross multiple tests."""

    stac_img = WNStacImage.from_tile(
        tile=test_data["stac_tile"],
        str_date=test_data["stac_date"],
        shape=test_data["stac_shape"],
    )

    assert stac_img.shape == test_data["stac_shape"]
    assert stac_img.name == test_data["stac_name"]

In [ ]:
rio.open?

Signature:
rio.open(
    fp,
    mode='r',
    driver=None,
    width=None,
    height=None,
    count=None,
    crs=None,
    transform=None,
    dtype=None,
    nodata=None,
    sharing=False,
    **kwargs,
)
Docstring:
Open a dataset for reading or writing.

The dataset may be located in a local file, in a resource located by
a URL, or contained within a stream of bytes.

In read ('r') or read/write ('r+') mode, no keyword arguments are
required: these attributes are supplied by the opened dataset.

In write ('w' or 'w+') mode, the driver, width, height, count, and dtype
keywords are strictly required.

Parameters
----------
fp : str, file object or pathlib.Path object
    A filename or URL, a file object opened in binary ('rb') mode,
    or a Path object.
mode : str, optional
    'r' (read, the default), 'r+' (read/write), 'w' (write), or
    'w+' (write/read).
driver : str, optional
    A short format driver name (e.g. "GTiff" or "JPEG") or a list of
    such names (see GDAL docs 